In [1]:
import pandas as pd
import numpy as np
import os
import json
import itertools as it
import time
import subprocess
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import OneHotEncoder
import tensorflow as tf
from datetime import date, datetime, timedelta
from tensorflow import keras
from sklearn.metrics import confusion_matrix,classification_report,ConfusionMatrixDisplay
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score, average_precision_score
from tensorflow.keras import layers, losses
from tensorflow.keras.models import Model
import warnings
warnings.filterwarnings("ignore")

2024-01-16 09:34:07.666430: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-01-16 09:34:07.697912: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-16 09:34:08.296106: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
majestic = pd.read_csv('./treino/majestic_ascii.csv')
majestic['69'] = 0
majestic['70'] = 0
majestic['71'] = 0
majestic

,domain,label,2,3,4,5,6,7,8,9,...,62,63,64,65,66,67,68,69,70,71
0,google.com,0,103,111,111,103,108,101,46,99,...,0,0,0,0,0,0,0,0,0,0
1,facebook.com,0,102,97,99,101,98,111,111,107,...,0,0,0,0,0,0,0,0,0,0
2,youtube.com,0,121,111,117,116,117,98,101,46,...,0,0,0,0,0,0,0,0,0,0
3,twitter.com,0,116,119,105,116,116,101,114,46,...,0,0,0,0,0,0,0,0,0,0
4,instagram.com,0,105,110,115,116,97,103,114,97,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999995,fidgit.com,0,102,105,100,103,105,116,46,99,...,0,0,0,0,0,0,0,0,0,0
999996,fixwindows10connections.com,0,102,105,120,119,105,110,100,111,...,0,0,0,0,0,0,0,0,0,0
999997,flyerguide.com,0,102,108,121,101,114,103,117,105,...,0,0,0,0,0,0,0,0,0,0
999998,forwardpathway.com,0,102,111,114,119,97,114,100,112,...,0,0,0,0,0,0,0,0,0,0


In [3]:
dga = pd.read_csv('./teste/df_teste3.csv')

In [4]:
dga = dga.drop('family', axis = 1)

In [5]:
dga.columns = majestic.columns

In [6]:
dga = dga.loc[~dga['domain'].isin(majestic['domain'])].reset_index(drop=True)

In [7]:
df_treino = pd.concat([majestic.sample(500000, random_state=1),dga], axis = 0, ignore_index=True)
df_treino

,domain,label,2,3,4,5,6,7,8,9,...,62,63,64,65,66,67,68,69,70,71
0,wwoof.fr,0,119,119,111,111,102,46,102,114,...,0,0,0,0,0,0,0,0,0,0
1,digimanthan.com,0,100,105,103,105,109,97,110,116,...,0,0,0,0,0,0,0,0,0,0
2,injo.com,0,105,110,106,111,46,99,111,109,...,0,0,0,0,0,0,0,0,0,0
3,aclanthology.info,0,97,99,108,97,110,116,104,111,...,0,0,0,0,0,0,0,0,0,0
4,aseabreeze.co.uk,0,97,115,101,97,98,114,101,101,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1790941,faxaxcagrmhrosmflhpy.com,1,102,97,120,97,120,99,97,103,...,0,0,0,0,0,0,0,0,0,0
1790942,inuqmdcivsmhpwyuyqdt.com,1,105,110,117,113,109,100,99,105,...,0,0,0,0,0,0,0,0,0,0
1790943,suudcivklyhovfkiaqvr.com,1,115,117,117,100,99,105,118,107,...,0,0,0,0,0,0,0,0,0,0
1790944,xjcysntwmphvificccgx.com,1,120,106,99,121,115,110,116,119,...,0,0,0,0,0,0,0,0,0,0


In [8]:
X_train = df_treino.drop(['domain','label'], axis = 1)
y_train = df_treino['label']

In [9]:
X_train = X_train.to_numpy()
y_train = y_train.to_numpy()

In [10]:
X_train.shape

(1790946, 70)

In [11]:
model_bin = tf.keras.models.load_model('best_model_fase3')

2024-01-16 09:34:14.858853: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-01-16 09:34:14.904884: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-01-16 09:34:14.905030: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [12]:
best = tf.keras.callbacks.ModelCheckpoint(
    filepath='best_model_fase4',
    save_weights_only=False,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

In [13]:
# Ajustando uso de memoria RAM e GPU
physical_devices = tf.config.list_physical_devices('GPU')
try:
  tf.config.experimental.set_memory_growth(physical_devices[0], True)
except:
  pass

In [14]:
model_bin.fit(X_train, y_train,
              epochs=5,
              callbacks = [best],
              validation_split = 0.2,
              batch_size=100)

Epoch 1/5


2024-01-16 09:34:17.770993: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8902
2024-01-16 09:34:18.263436: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:606] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2024-01-16 09:34:18.265750: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7f291c015940 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-01-16 09:34:18.265764: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3050 Laptop GPU, Compute Capability 8.6
2024-01-16 09:34:18.269640: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:255] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-01-16 09:34:18.351261: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifeti

14328/14328 [==============================] - ETA: 0s - loss: 0.0312 - accuracy: 0.9895INFO:tensorflow:Assets written to: best_model_fase4/assets


INFO:tensorflow:Assets written to: best_model_fase4/assets


14328/14328 [==============================] - 347s 24ms/step - loss: 0.0312 - accuracy: 0.9895 - val_loss: 0.0493 - val_accuracy: 0.9810
Epoch 2/5
14328/14328 [==============================] - 321s 22ms/step - loss: 0.0249 - accuracy: 0.9917 - val_loss: 0.0551 - val_accuracy: 0.9796
Epoch 3/5
14328/14328 [==============================] - 321s 22ms/step - loss: 0.0206 - accuracy: 0.9934 - val_loss: 0.0793 - val_accuracy: 0.9703
Epoch 4/5
14328/14328 [==============================] - 323s 23ms/step - loss: 0.0175 - accuracy: 0.9945 - val_loss: 0.0702 - val_accuracy: 0.9758
Epoch 5/5
14328/14328 [==============================] - 322s 22ms/step - loss: 0.0148 - accuracy: 0.9955 - val_loss: 0.0666 - val_accuracy: 0.9781


In [15]:
model_bin.load_weights('best_model_fase4')

2024-01-16 10:01:29.909663: W tensorflow/core/util/tensor_slice_reader.cc:97] Could not open best_model_fase4: FAILED_PRECONDITION: best_model_fase4; É um diretório: perhaps your file is in a different file format and you need to use a different restore operator?


In [16]:
legit = majestic.loc[~majestic['domain'].isin(df_treino['domain'])].reset_index(drop=True)

## Teste 4

In [17]:
dga = pd.read_csv('./teste/df_teste4.csv')

In [18]:
dga = dga.drop('family', axis = 1)

In [19]:
dga.columns = legit.columns

In [20]:
df_teste = pd.concat([legit,dga], axis = 0, ignore_index=True)

In [21]:
X_test = df_teste.drop(['domain','label'], axis = 1)
y_test = df_teste['label']

In [22]:
X_test = X_test.to_numpy()
y_test = y_test.to_numpy()

In [23]:
X_test.shape

(1961827, 70)

In [24]:
with tf.device('/cpu:0'):
    predictions = model_bin.predict(X_test)
    predictions = np.where(predictions > 0.5, 1, 0)
    
print('Matriz de Confusão')
print(confusion_matrix(y_test,predictions))

print('Relatório de Classificação')
print(classification_report(y_test,predictions, digits=4))

61308/61308 [==============================] - 823s 13ms/step
Matriz de Confusão
[[ 485231   14769]
 [  37322 1424505]]
Relatório de Classificação
              precision    recall  f1-score   support

           0     0.9286    0.9705    0.9491    500000
           1     0.9897    0.9745    0.9820   1461827

    accuracy                         0.9734   1961827
   macro avg     0.9592    0.9725    0.9656   1961827
weighted avg     0.9742    0.9734    0.9736   1961827



## Teste 5

In [25]:
dga = pd.read_csv('./teste/df_teste5.csv')

In [26]:
dga = dga.drop('family', axis = 1)

In [27]:
dga.columns = legit.columns

In [28]:
df_teste = pd.concat([legit,dga], axis = 0, ignore_index=True)

In [29]:
X_test = df_teste.drop(['domain','label'], axis = 1)
y_test = df_teste['label']

In [30]:
X_test = X_test.to_numpy()
y_test = y_test.to_numpy()

In [31]:
X_test.shape

(1795392, 70)

In [32]:
with tf.device('/cpu:0'):
    predictions = model_bin.predict(X_test)
    predictions = np.where(predictions > 0.5, 1, 0)
    
print('Matriz de Confusão')
print(confusion_matrix(y_test,predictions))

print('Relatório de Classificação')
print(classification_report(y_test,predictions, digits=4))

56106/56106 [==============================] - 694s 12ms/step
Matriz de Confusão
[[ 485231   14769]
 [  19834 1275558]]
Relatório de Classificação
              precision    recall  f1-score   support

           0     0.9607    0.9705    0.9656    500000
           1     0.9886    0.9847    0.9866   1295392

    accuracy                         0.9807   1795392
   macro avg     0.9746    0.9776    0.9761   1795392
weighted avg     0.9808    0.9807    0.9808   1795392



## Teste 6

In [33]:
dga = pd.read_csv('./teste/df_teste6.csv')

In [34]:
dga = dga.drop('family', axis = 1)

In [35]:
dga.columns = legit.columns

In [36]:
df_teste = pd.concat([legit,dga], axis = 0, ignore_index=True)

In [37]:
X_test = df_teste.drop(['domain','label'], axis = 1)
y_test = df_teste['label']

In [38]:
X_test = X_test.to_numpy()
y_test = y_test.to_numpy()

In [39]:
X_test.shape

(1946427, 70)

In [40]:
with tf.device('/cpu:0'):
    predictions = model_bin.predict(X_test)
    predictions = np.where(predictions > 0.5, 1, 0)
    
print('Matriz de Confusão')
print(confusion_matrix(y_test,predictions))

print('Relatório de Classificação')
print(classification_report(y_test,predictions, digits=4))

60826/60826 [==============================] - 733s 12ms/step
Matriz de Confusão
[[ 485231   14769]
 [  23129 1423298]]
Relatório de Classificação
              precision    recall  f1-score   support

           0     0.9545    0.9705    0.9624    500000
           1     0.9897    0.9840    0.9869   1446427

    accuracy                         0.9805   1946427
   macro avg     0.9721    0.9772    0.9746   1946427
weighted avg     0.9807    0.9805    0.9806   1946427

